# Initial Setup

Mentions: 
- Thank you for emasters for their excellent [tutorial](https://www.youtube.com/watch?v=DLngCtsG3bk)

In [ ]:
# Check for Nvidia GPU. Required for cudaNN
!nvidia-smi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
import math
import decimal
import copy
import glob

In [ ]:
#True if currently trainging to detect mask
maskDetectionTrain = False

#0 - fullFaceArea (treat mask as part of person's identity), 1 - detect if there's a mask, 2 - detect top half of face, 3 - detect eye region
labelNames = 0

#backup file location
backupLocationPath = "/content/drive/MyDrive/Training/maskmodel"

# Data Prep

### Clone Project Repo with modified Darknet
!git clone https://github.com/HughN7/darknet

In [ ]:
#Compile Darknet
!make

In [ ]:
#Configure yolov3.cfg
if(maskDetectionTrain):
    !cp cfg/yolov3.cfg cfg/yolov3_training.cfg #copy config into train

    !sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
    !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
    !sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg

    !sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
    !sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
    !sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg

    #1 class at 18 filters while 2 classes at 21
    !sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
    !sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
    !sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg
else:
    !cp cfg/yolov3.cfg cfg/yolov3_training.cfg #Copy two files

    !sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
    !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
    !sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg

    !sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
    !sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
    !sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg

    #1 class at 18 filters while 2 classes at 21
    !sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
    !sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
    !sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
if(labelNames == 0):
    !echo -e 'User' > data/obj.names #.names save to obj.names
if(labelNames == 1):
    !echo -e 'Mask\nNoMask' > data/obj.names #.names save to obj.names
if(labelNames == 2):
    !echo -e 'UserTopArea' > data/obj.names #.names save to obj.names
if(labelNames == 3):
    !echo -e 'UserEyeArea' > data/obj.names #.names save to obj.names
    

In [ ]:
#.data = 5 lines:
# number of classes | classes= 2\n
# path to train.txt | train = data/train.txt\n
# path to test.txt | valid  = data/test.txt\n
# path to obj.names | names = data/obj.names\n
# path to save trained weights = backup = 
if(maskDetectionTrain):
    !echo -e 'classes= 2\ntrain = data/train.txt\nvalid = data/train.txt\nnames = data/obj.names\nbackup = ' + str(backupLocationPath) + '' > data/obj.data
else:
    !echo -e 'classes= 1\ntrain = data/train.txt\nvalid = data/train.txt\nnames = data/obj.names\nbackup = ' + str(backupLocationPath) + '' > data/obj.data

In [ ]:
#unzip training data
!mkdir data/obj
!unzip /content/drive/MyDrive/SeniorProject/MaskTrainingFiles/masks.zip -d data/obj

# Training

### Create train.txt

In [ ]:
#Tell which images + path that yolo needs to be trained on
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

### Download Pre-trained weights

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

### Execute

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 2>&1 > /content/drive/MyDrive/SeniorProject/maskmodel/log.txt -dont_show -map

In [ ]:
#use this in the event colab kicks you out (free tier)
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/SeniorProject/maskmodel/yolov3_training_last.weights > log.txt -dont_show -map